<a href="https://colab.research.google.com/github/toblaroni/Numerical-Computation-Methods/blob/main/numCompMethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Gaussian elimination

In [ ]:
def lower_triangular_solve(A, b):
    # Make sure the arrays are stored with correct type
    A = A.astype(np.float64)
    b = b.astype(np.float64)
    
    # Check A is lower triangular
    rows = A.shape[0]
    cols = A.shape[1]
    
    x = np.zeros(b.shape)
    
    for i in range(0, rows):
        for j in range(i + 1, cols):
            if (j > i and not np.isclose(A[i, j], 0)):
                print("Array A is not lower triangular")
                return
            elif j == i and np.isclose(A[i, j], 0):
                print("Array A is not lower triangular1")
                return
            
    # Solve the system
    for i in range(0, rows):
        rA = 1 / A[i][i]
        sum = 0
        for j in range(0, i):
            sum += A[i][j] * x[j]
        x[i] = rA * (b[i] - sum)
    return x

In [ ]:
def upper_triangular_solve(A, b):
    # Make sure array is stored correctly
    A = A.astype(np.float64)
    b = b.astype(np.float64)
    
    # Make sure the array is square
    n, m = A.shape
    if (n != m):
        print("A is not square")
    
    # Make sure it's lower triangular
    for i in range(0, n):
        for j in range(0, i + 1):
            if j < i and not np.isclose(A[i, j], 0):
                print("A is not upper triangular")
                return
            elif j == i and np.isclose(A[i, j], 0):
                print("A is not upper triangular")
                return
            
    x = np.empty_like(b)
    
    # Solve by backward substitution
    x[n - 1] = b[n - 1] / A[n - 1, m - 1]
    for i in range(2, n+1):
        x[n-i] = b[n-i] / A[n-i, n-i]
        for j in range(n-i+1, m):
            x[n-i] -= A[n-i,j]*x[j] / A[n-i, n-i]
    return x

In [ ]:
def gauss_elimination(A, b):
    # Convert to proper precision
    A = A.astype(np.float64)
    b = b.astype(np.float64)
    
    # Check shape and diagonals
    n, m = A.shape
    if not n == m:
        raise ValueError(f"A is not square matrix. n={n}, m={m}")
    for i in range(0, n):
        if (A[i, i] == 0):
            raise ValueError(f"Non-zero diagonal value at index {i}")
            
    # Perform forward elimination
    for j in range(n):
        for i in range(j + 1, n):
            factor = A[i, j] / A[j, j]
            b[i] -= factor * b[j]
            for k in range(0, n):
                A[i, k] -= factor * A[j, k]

    return upper_triangular_solve(A, b)

# Iterative Methods

In [ ]:
def Jacobi_iteration(A, b, max_iteration, x0=None):
    A = A.astype(np.float64)
    b = b.astype(np.float64)
    
    n = len(b)
    
    for i in range(n):
        if np.isclose(A[i, i], 0):
            print("Diagonal element is non zero")
            return
    
    P = np.zeros([n, n])
    p = np.zeros(n)
    for i in range(n):
        # Compute P and p
        p[i] = b[i] / A[i, i]
        for j in range(n):
            P[i, j] = A[i, j] / A[i, i]
        P[i, i] = 0

    if x0 is None:
        x = np.zeros_like(b)
    else:
        x = x0.copy()

    # Compute
    x = np.zeros(n)
    for it in range(max_iteration):
        x = p - np.matmul(P, x)
    
    return x

In [ ]:
# Test
A = np.array([[2, 1], [1, 2]])
b = np.array([3, 3])
x_exact = np.array([1,1])

myX = Jacobi_iteration(A, b, 100)
print('my solution = ', myX)


my solution =  [1. 1.]


In [ ]:
def Gauss_Seidel_iteration(A, b, max_iteration, x0=None):
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    n, m = A.shape
    
    for i in range(n):
        if np.isclose(A[i, i], 0):
            print("Diagonal element (%f %f) is zero!" %(i, i))
            return
        
    LD = np.zeros_like(A)
    U = np.zeros_like(A)
    for i in range(n):
        for j in range(n):
            if i < j:
                U[i, j] = A[i, j]
            else:
                LD[i, j] = A[i, j]

    p = lower_triangular_solve(LD, b)
    
    if x0 is None:
        x = np.zeros_like(b)
    else:
        x = x0.copy()

    # Array to hold the results
    Ux = np.empty_like(x)
    for it in range(max_iteration):
        for i in range(n):
            Ux[i] = 0.0
            for j in range(i+1, n):
                Ux[i] += U[i, j] * x[j]
        Px = lower_triangular_solve(LD, Ux)
        x = p - Px
    return x

In [ ]:
# Test different linear solvers starting from the above two-dimensional linear system
A = np.array([[2, 1], [1, 2]])
b = np.array([3, 3])
x_exact = np.array([1,1])

# numpy linear solver
x0 = np.linalg.solve(A,b)
print("Solution by numpy solver:", x0)

x = Jacobi_iteration(A, b, 40)
print("Solution by Jacobi iteration: ",x)
print("Error: ", x - x_exact)
print("Residual: ", np.matmul(A,x)-b)

x = Gauss_Seidel_iteration(A, b, 4)
print("Solution by Gauss Seidel iteration: ",x)
print("Error: ", x - x_exact)
print("Residual: ", np.matmul(A,x)-b)

Solution by numpy solver: [1. 1.]
Solution by Jacobi iteration:  [1. 1.]
Error:  [-9.09494702e-13 -9.09494702e-13]
Residual:  [-2.72848411e-12 -2.72848411e-12]
Solution by Gauss Seidel iteration:  [1.0078125  0.99609375]
Error:  [ 0.0078125  -0.00390625]
Residual:  [0.01171875 0.        ]


# Gaussian Elimination With pivoting

In [ ]:
def Gaussian_elimination_pivoting(A, b):
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    n = len(b)

    for i in range(n):
        maximum = A[i, i]
        max_index = i
        for j in range(i+1, n):
            if abs(A[j, i]) > maximum:
                maximum = abs(A[j, i])
                max_index = j

        # Swap rows if necessary
        if max_index is not i:
            for j in range(n):
                tmp = A[i, j]
                A[i, j] = A[max_index, j]
                A[max_index, j] = tmp
            tmp = b[i]
            b[i] = b[max_index]
            b[max_index] = tmp
        
        # Gaussian elimination to create upper triangular matrix
        for j in range(i + 1, n):
            factor = A[j, i] / A[i, i]
            b[j] -= factor * b[i]
            for k in range(0, n):
                A[j, k] -= factor * A[i, k]

    return upper_triangular_solve(A, b)

In [ ]:
# Testing
A = np.array([[-10, 2, 0, 67],
              [-2, 50, -77, 1.e-5],
              [1, 7, 30, 8],
              [-100, -7, 0.001, 80]])
b = np.array([1, 2, 9, 0])

# numpy linear solvers
x0 = np.linalg.solve(A,b)
print("Solution by numpy solver:", x0)

x = Gaussian_elimination_pivoting(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", np.matmul(A,x)-b)

Solution by numpy solver: [-2.58309032e-02  3.69501922e-01  2.14633220e-01  4.01063176e-05]
Gaussian elimination:  [-2.58309032e-02  3.69501922e-01  2.14633220e-01  4.01063176e-05]
Residual:  [0.00000000e+00 3.55271368e-15 0.00000000e+00 0.00000000e+00]


# Sparse matrix multiplication

In [ ]:
import numpy as np

A_real = np.array([3, 1.5, 4, 4, 2.2, 2, 6, 4, -9], dtype=np.float64)
I_row = np.array([0,   0,  1, 1,  1,  2, 2, 3, 3], dtype=np.int32)
I_col = np.array([0,   2,  0, 1,  2,  1, 2, 2, 3], dtype=np.int32) 

nonzero = len(A_real)
dim = 4
y = np.zeros(dim) + 1.
z = np.zeros(dim)

for k in range(nonzero):
    z[I_row[k]] += A_real[k] * y[I_col[k]]

print(z)

[ 4.5 10.2  8.  -5. ]


# Euler's Method

In [ ]:
def Euler_method(t0, d0, dt, n):
    d = np.zeros(n+1)
    d[0] = d0

    t = np.zeros(n+1)
    for i in range(n+1):
        t[i] = t0 + i * dt

    # Compute Euler's method
    for it in range(1, n+1):
        d[it] = d[it - 1] + dt * f(t[it - 1], d[it - 1])

    return t, d


# Midpoint method

In [ ]:
def midpoint_method(t0, d0, dt, n):
    d = np.zeros(n+1)
    d[0] = d0

    t = np.zeros(n+1)
    for i in range(n+1):
        t[i] = t0 + dt*i
    
    for i in range(1, n+1):
        d_mid = d[i-1] + 0.5 * dt * f(t[i-1], d[i-1])
        t_mid = t[i-1] + dt*0.5
        d[i] = d[i-1] + dt * f(t_mid, d_mid)

    return t, d

In [ ]:
def f(t, d):
    return 2 * np.sqrt(d)

def d_exact(t):
    return t**2

t0 = float(0.1)
d0 = float(0.001)
dt = float(0.1)
n = int(10)

t_Euler, d_Euler = Euler_method(t0, d0, dt, n)
t_midpoint, d_midpoint = midpoint_method(t0, d0, dt, n)


de = np.zeros(n+1)
for i in range(n+1):
    de[i] = d_exact(t_Euler[i])
    

print(d_Euler)
# import matplotlib.pyplot as plt
# fig = plt.figure()
# plt.plot(t_Euler, d_Euler, "-b", label="Euler")
# plt.plot(t_Euler, de, "-r", label="Exact")
# plt.plot(t_midpoint, d_midpoint, "--g", label="midpoint")
# plt.legend(loc="upper left")
# plt.xlabel("t")
# plt.ylabel("d")
# plt.grid()
# plt.show()

[0.001      0.00732456 0.02444128 0.05570869 0.10291407 0.16707452
 0.24882401 0.34858854 0.46667131 0.603298   0.75864253]


# Bisection and Newton Methods

In [ ]:
def newton(f, df, x0, tol, n):
    x = x0
    y = f(x)
    it = 0
    while abs(y) > tol and it < n:
        it += 1
        x -= y / df(x)
        y = f(x)
    return x, it

def bisection(f, x0, x1, tol, n):
    it = 0
    x = (x0 + x1) / 2.
    xL = x0
    xR = x1

    y = f(x)
    yL = f(xL)
    yR = f(xR)
    while abs(y) > tol and it < n:
        if (it > 10000):
            print("Bisection Method failed to converge")
            return None, None
        if y*yL < 0:
            xR = x
            yR = y
        elif y * yR < 0:
            xL = x
            yL = y
        x = (xL + xR) / 2
        y = f(x)
        it += 1
    
    return x, it

In [ ]:
def f(t):
    return t*t*t - 2.0

def df(t):
    return 3.*t*t


x, it = newton(f, df, 2.0, 1.e-6, 10)
print(f"Newton's method: {x} after {it} iterations")

x, it = bisection(f, 0.0, 2.0, 1.e-6, 100)
print(f"Bisection method: {x} after {it} iterations")

Newton's method: 1.2599210498953948 after 5 iterations
Bisection method: 1.2599210739135742 after 20 iterations
